In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import requests
from jsonmerge import merge
from pprint import pprint
import pandas as pd
from pandas.io.json import json_normalize
import json
import io

pd.set_option('display.max_columns', None)

In [2]:
URL = "https://developers.zomato.com/api/v2.1/cities?q=las%20vegas"
header = {"User-agent": "curl/7.43.0", "Accept": "application/json", "user_key": "5978814818e9908f86073519a08ca500"}

response = requests.get(URL, headers=header)
zomato_api = '5978814818e9908f86073519a08ca500'

In [3]:
def get_location_details(query):

    headers = {
        'Accept': 'application/json',
        'user-key': zomato_api,
    }
    params = (
        ('query', query),
    )

    response = requests.get('https://developers.zomato.com/api/v2.1/locations', headers=headers, params=params)
    data = response.json()

    for loc in data['location_suggestions']:
        loc_id = loc['entity_id']
        loc_type = loc['entity_type']

    return loc_id, loc_type

In [4]:
def get_restaurants(ent_id, ent_type):

    headers = {
        'Accept': 'application/json',
        'user-key': zomato_api,
    }

    params = (
        ('entity_id', ent_id),
        ('entity_type', ent_type),
    )

    response = requests.get('https://developers.zomato.com/api/v2.1/search', headers=headers, params=params)

    return response.json()

In [5]:
entity_id, entity_type = get_location_details("Las Vegas")
print(entity_id)
print(entity_type)

282
city


In [6]:
zomato_data = get_restaurants(entity_id, entity_type)
zomato_data

{'results_found': 5397,
 'results_start': 0,
 'results_shown': 20,
 'restaurants': [{'restaurant': {'R': {'res_id': 16979446,
     'is_grocery_store': False,
     'has_menu_status': {'delivery': -1, 'takeaway': -1}},
    'apikey': '5978814818e9908f86073519a08ca500',
    'id': '16979446',
    'name': 'Mon Ami Gabi',
    'url': 'https://www.zomato.com/las-vegas/mon-ami-gabi-las-vegas?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1',
    'location': {'address': '3655 Las Vegas Boulevard South 89109',
     'locality': 'The Strip',
     'city': 'Las Vegas',
     'city_id': 282,
     'latitude': '36.1121300000',
     'longitude': '-115.1694200000',
     'zipcode': '89109',
     'country_id': 216,
     'locality_verbose': 'The Strip, Las Vegas'},
    'switch_to_order_menu': 0,
    'cuisines': 'Breakfast, French',
    'timings': '7 AM to 10 PM (Mon-Thu, Sun), 7 AM to 11 PM (Fri-Sat)',
    'average_cost_for_two': 70,
    'price_range': 4,
    'currency': '$',
    'highlights': ['Lunc

In [7]:
json_normalize(zomato_data['restaurants']).head()

,restaurant.R.has_menu_status.delivery,restaurant.R.has_menu_status.takeaway,restaurant.R.is_grocery_store,restaurant.R.res_id,restaurant.all_reviews.reviews,restaurant.all_reviews_count,restaurant.apikey,restaurant.average_cost_for_two,restaurant.book_again_url,restaurant.book_form_web_view_url,restaurant.cuisines,restaurant.currency,restaurant.deeplink,restaurant.establishment,restaurant.establishment_types,restaurant.events_url,restaurant.featured_image,restaurant.has_online_delivery,restaurant.has_table_booking,restaurant.highlights,restaurant.id,restaurant.include_bogo_offers,restaurant.is_book_form_web_view,restaurant.is_delivering_now,restaurant.is_table_reservation_supported,restaurant.is_zomato_book_res,restaurant.location.address,restaurant.location.city,restaurant.location.city_id,restaurant.location.country_id,restaurant.location.latitude,restaurant.location.locality,restaurant.location.locality_verbose,restaurant.location.longitude,restaurant.location.zipcode,restaurant.medio_provider,restaurant.menu_url,restaurant.mezzo_provider,restaurant.name,restaurant.offers,restaurant.opentable_support,restaurant.phone_numbers,restaurant.photo_count,restaurant.photos_url,restaurant.price_range,restaurant.store_type,restaurant.switch_to_order_menu,restaurant.thumb,restaurant.timings,restaurant.url,restaurant.user_rating.aggregate_rating,restaurant.user_rating.rating_color,restaurant.user_rating.rating_obj.bg_color.tint,restaurant.user_rating.rating_obj.bg_color.type,restaurant.user_rating.rating_obj.title.text,restaurant.user_rating.rating_text,restaurant.user_rating.votes
0,-1,-1,False,16979446,"[{'review': []}, {'review': []}, {'review': []...",319,5978814818e9908f86073519a08ca500,70,,,"Breakfast, French",$,zomato://restaurant/16979446,[],[],https://www.zomato.com/las-vegas/mon-ami-gabi-...,https://b.zmtcdn.com/data/res_imagery/16979446...,0,0,"[Lunch, Serves Alcohol, Breakfast, Dinner, Out...",16979446,True,0,0,0,0,3655 Las Vegas Boulevard South 89109,Las Vegas,282,216,36.1121300000,The Strip,"The Strip, Las Vegas",-115.1694200000,89109,False,https://www.zomato.com/las-vegas/mon-ami-gabi-...,OTHER,Mon Ami Gabi,[],0,(702) 944-4224,241,https://www.zomato.com/las-vegas/mon-ami-gabi-...,4,,0,https://b.zmtcdn.com/data/res_imagery/16979446...,"7 AM to 10 PM (Mon-Thu, Sun), 7 AM to 11 PM (F...",https://www.zomato.com/las-vegas/mon-ami-gabi-...,4.6,3F7E00,800,lime,4.6,Excellent,1214
1,-1,-1,False,16982906,"[{'review': []}, {'review': []}, {'review': []...",285,5978814818e9908f86073519a08ca500,40,,,"Burger, Fast Food",$,zomato://restaurant/16982906,[Casual Dining],[],https://www.zomato.com/las-vegas/gordon-ramsay...,https://b.zmtcdn.com/data/res_imagery/16982906...,0,0,"[No Alcohol Available, Dinner, Debit Card, Cas...",16982906,True,0,0,0,0,3667 South Las Vegas Boulevard 89109,Las Vegas,282,216,36.1106106000,The Strip,"The Strip, Las Vegas",-115.1722684000,89109,False,https://www.zomato.com/las-vegas/gordon-ramsay...,OTHER,Gordon Ramsay Burger,[],0,"(702) 785-5555, (702) 785-5462",416,https://www.zomato.com/las-vegas/gordon-ramsay...,3,,0,https://b.zmtcdn.com/data/res_imagery/16982906...,"10:30 AM to 12 Midnight (Mon-Fri), 12 Midnight...",https://www.zomato.com/las-vegas/gordon-ramsay...,4.5,3F7E00,700,lime,4.5,Excellent,608
2,-1,-1,False,16979203,"[{'review': []}, {'review': []}, {'review': []...",157,5978814818e9908f86073519a08ca500,25,,,"Asian, Thai, Vegetarian",$,zomato://restaurant/16979203,[],[],https://www.zomato.com/las-vegas/lotus-of-siam...,https://b.zmtcdn.com/data/res_imagery/16979203...,0,0,"[Dinner, Lunch, Serves Alcohol, Indoor Seating...",16979203,True,0,0,0,0,"953 E Sahara Ave, Las Vegas 89104",Las Vegas,282,216,36.1421289000,Winchester,"Winchester, Las Vegas",-115.1423943000,89104,False,https://www.zomato.com/las-vegas/lotus-of-siam...,OTHER,Lotus of Siam,[],0,(702) 735-3033,156,https://www.zomato.com/las-vegas/lotus-of-siam...,2,,0,https://b.zmtcdn.com/data/res_imagery/16979203...,"11:30 AM to 2:30 PM, 5:30 PM t

In [16]:
zomato_df = json_normalize(zomato_data['restaurants'])
cols = ['restaurant.name', 'restaurant.location.address', 'restaurant.all_reviews_count',  'restaurant.user_rating.aggregate_rating', 'restaurant.user_rating.votes' ]
zomato_relevant1 = zomato_df[cols]
zomato_relevant1.head(5)

,restaurant.name,restaurant.location.address,restaurant.all_reviews_count,restaurant.user_rating.aggregate_rating,restaurant.user_rating.votes
0,Mon Ami Gabi,3655 Las Vegas Boulevard South 89109,319,4.6,1214
1,Gordon Ramsay Burger,3667 South Las Vegas Boulevard 89109,285,4.5,608
2,Lotus of Siam,"953 E Sahara Ave, Las Vegas 89104",157,4.6,606
3,Bouchon Bistro - The Venetian,"The Venetian, 3355 Las Vegas Boulevard South, ...",175,4.6,693
4,Mesa Grill,3570 South Las Vegas Boulevard 89109,222,4.3,812


In [10]:
def get_restaurants2(ent_id, ent_type, start_count):

    headers = {
        'Accept': 'application/json',
        'user-key': zomato_api,
    }

    params = (
        ('entity_id', ent_id),
        ('entity_type', ent_type),
        ('start', start_count)
    )

    response = requests.get('https://developers.zomato.com/api/v2.1/search', headers=headers, params=params)

    return response.json()

In [17]:
zomato_data2 = get_restaurants2(entity_id, entity_type, 20)
zomato_df2 = json_normalize(zomato_data2['restaurants'])
zomato_relevant2 = zomato_df2[cols]
zomato_relevant2.head(5)

,restaurant.name,restaurant.location.address,restaurant.all_reviews_count,restaurant.user_rating.aggregate_rating,restaurant.user_rating.votes
0,Spago,3500 Las Vegas Boulevard. South 89109,74,4.7,354
1,L'Atelier de Joël Robuchon,"3799 Las Vegas Boulevard S, Las Vegas 89109",75,4.9,224
2,Bachi Burger,"470 East Windmill Lane, Suite 100, South Las V...",81,4.4,259
3,Jean-Philippe Pâtisserie,"3600 Las Vegas Boulevard S, Las Vegas 89109",83,4.9,237
4,Hot N Juicy Crawfish,"4810 Spring Mountain Road, Las Vegas 89102",49,4.6,261


In [18]:
zomato_data3 = get_restaurants2(entity_id, entity_type, 40)
zomato_df3 = json_normalize(zomato_data3['restaurants'])
zomato_relevant3 = zomato_df3[cols]

In [19]:
zomato_data4 = get_restaurants2(entity_id, entity_type, 60)
zomato_df4 = json_normalize(zomato_data4['restaurants'])
zomato_relevant4 = zomato_df4[cols]

In [20]:
# last pull, 100 record limit
zomato_data5 = get_restaurants2(entity_id, entity_type, 80)
zomato_df5 = json_normalize(zomato_data5['restaurants'])
zomato_relevant5 = zomato_df5[cols]

In [23]:
zomato_concat = [zomato_relevant1, zomato_relevant2, zomato_relevant3, zomato_relevant4, zomato_relevant5]
zomato_100 = pd.concat(zomato_concat)
zomato_100.rename(columns={"restaurant.name": "name", 
                           'restaurant.location.address':'address',
                           'restaurant.all_reviews_count':'zomato_review_count',
                           'restaurant.user_rating.aggregate_rating':'aggregate_rating',
                           'restaurant.user_rating.votes':'user_rating_votes'
                        },
                inplace = True)
zomato_100.head(5)

,name,address,zomato_review_count,aggregate_rating,user_rating_votes
0,Mon Ami Gabi,3655 Las Vegas Boulevard South 89109,319,4.6,1214
1,Gordon Ramsay Burger,3667 South Las Vegas Boulevard 89109,285,4.5,608
2,Lotus of Siam,"953 E Sahara Ave, Las Vegas 89104",157,4.6,606
3,Bouchon Bistro - The Venetian,"The Venetian, 3355 Las Vegas Boulevard South, ...",175,4.6,693
4,Mesa Grill,3570 South Las Vegas Boulevard 89109,222,4.3,812


In [29]:
zomato_100.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 19
Data columns (total 5 columns):
name                   100 non-null object
address                100 non-null object
zomato_review_count    100 non-null int64
aggregate_rating       100 non-null object
user_rating_votes      100 non-null int64
dtypes: int64(2), object(3)
memory usage: 4.7+ KB


In [31]:
zomato_100['aggregate_rating'] = zomato_100['aggregate_rating'].astype(str).astype(float)

In [32]:
zomato_100.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 19
Data columns (total 5 columns):
name                   100 non-null object
address                100 non-null object
zomato_review_count    100 non-null int64
aggregate_rating       100 non-null float64
user_rating_votes      100 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.7+ KB


In [33]:
zomato_100.isnull().values.any()

False

In [ ]:
zomato_100.to_csv(r'C:\Users\willm\OneDrive\Documents\MSCA\31012_Data_Engineering\Final_Project\Datasets\Yelp\zomato_100.csv')